In [2]:
pip install simplet5

# pip install datasets

SyntaxError: invalid syntax (1032310542.py, line 1)

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from simplet5 import SimpleT5
from tabulate import tabulate
import nltk
from datetime import datetime
import numpy as np

In [4]:
pip install simplet5


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 1.3 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 3.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 3.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 4.1 MB/s eta 0:00:0000:0100:01
  Created wheel for simplet5: filename=simplet5-0.1.4-py3-none-any.whl size=6857 sha256=ca28b9ac6232f4068de6f574f7d23fc2ada009be9f67b3bebafe46dfffed1fac
  Stored in directory: /root/.cache/pip/wheels/b4/7d/af/743765400878438a7593f13f89fdf4004dcde0f2a8e6cb6684
Successfully built simplet5
  Attempting uninstall: setuptools
    Found existing installation: setuptools 68.0.0
    Uninstalling setuptools-68.0.0:
      Successfully uninstalled setuptools-68.0.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.33.0
    Uninstalling transformers-4.33.0:
      Succe

In [5]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [7]:
import datasets
answersummData = datasets.load_dataset("alexfabbri/answersumm")


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/alexfabbri--answersumm-18f7fb4735693092/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
answersummDataTest = answersummData["test"]
answersummDataTrain = answersummData["train"]
answersummDataVal = answersummData["validation"]

In [9]:
answersummDataTest

Dataset({
    features: ['answers', 'question', 'example_id', 'summaries', 'mismatch_info', 'annotator_id', 'cluster_summaries'],
    num_rows: 1000
})

In [10]:
answersummDataTest[2]['question']['title']

'Buying vs catching a horse in Red Dead Redemption'

In [11]:
def flatten(example):
  answers = example["answers"]
#  print(answers[0])
  allanswers = ""
  for answer in answers:
    lines = answer["sents"]
    for line in lines:
      allanswers = allanswers + line["text"] + " "
    
  return {
      "title": example["question"]["title"],
      "question": example["question"]["question"],
      "allanswers": allanswers,
      "clustersummary": " ".join(example["cluster_summaries"][0]),
      "firstsummary": example["summaries"][0][0],
      "secondsummary": example["summaries"][0][1],
      "question+clustersumm": example["question"]["question"] + " " + " ".join(example["cluster_summaries"][0]),
      "question+allanswers": example["question"]["question"] + " " + allanswers
      }

In [12]:
train_data_txt = answersummDataTrain.map(flatten, remove_columns=['answers', 'question', 'example_id', 'summaries', 'mismatch_info', 'annotator_id', 'cluster_summaries'])
val_data_txt = answersummDataVal.map(flatten, remove_columns=['answers', 'question', 'example_id', 'summaries', 'mismatch_info', 'annotator_id', 'cluster_summaries'])
test_data_txt = answersummDataTest.map(flatten, remove_columns=['answers', 'question', 'example_id', 'summaries', 'mismatch_info', 'annotator_id', 'cluster_summaries'])


  0%|          | 0/2783 [00:00<?, ?ex/s]

  0%|          | 0/500 [00:00<?, ?ex/s]

  0%|          | 0/1000 [00:00<?, ?ex/s]

In [13]:
train_data_txt

Dataset({
    features: ['question', 'title', 'allanswers', 'clustersummary', 'firstsummary', 'secondsummary', 'question+clustersumm', 'question+allanswers'],
    num_rows: 2783
})

In [14]:
train_data_txt.column_names

['question',
 'title',
 'allanswers',
 'clustersummary',
 'firstsummary',
 'secondsummary',
 'question+clustersumm',
 'question+allanswers']

In [15]:
def average_token_len(data):
  sum = 0
  for i in range(len(data)):
    sum += len(data[i].split())
  return sum/len(data)


In [16]:
print(average_token_len(train_data_txt['question']))
print(average_token_len(train_data_txt['allanswers']))
print(average_token_len(train_data_txt['firstsummary']))
print(average_token_len(train_data_txt['secondsummary']))
print(average_token_len(train_data_txt['clustersummary']))
print(average_token_len(train_data_txt['question+allanswers']))

52.54293927416457
685.0416816385195
48.361480416816384
41.03305785123967
50.11749910168883
737.5846209126842


In [17]:
# Model-Bart trained on X-Sum

# model_name = "sshleifer/distilbart-xsum-12-3"
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

In [18]:
# Model T-5
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:220: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  return len(token_ids_0 + eos) * [0]
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be use

In [19]:
val_data_txt[0]["question+clustersumm"]

"Assume that someone (friend, family) asks you to format his/her CV. They typically might want to apply to a job where text processing etc. is not one of the key requirements and they ask you to do it in order to have a nice result and/or to win time, although they could probably do it themselves in Word.  Can it be negative for them? Imagine for instance that the interviewer asks them how they made such a nice CV. It is generally agreed that it's perfectly acceptable to help someone format their CV and it's unlikely to have any negative repercussions for them."

In [20]:
"""### **Parameters**"""

encoder_max_length = 512
decoder_max_length = 128
input = "clustersummary"
output = "secondsummary"

training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=5,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=2,  # demo
    per_device_eval_batch_size=2,
    # learning_rate=3e-05,
    warmup_steps=100,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=50,
    save_total_limit=3,
)

In [21]:
def batch_tokenize_preprocess(batch, tokenizer, max_source_length, max_target_length):
    source, target = batch[input], batch[output]
    source_tokenized = tokenizer(
        source, padding="max_length", truncation=True, max_length=max_source_length
    )
    target_tokenized = tokenizer(
        target, padding="max_length", truncation=True, max_length=max_target_length
    )

    batch = {k: v for k, v in source_tokenized.items()}
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch


In [22]:
train_data = train_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=train_data_txt.column_names,
)


  0%|          | 0/3 [00:00<?, ?ba/s]

In [23]:
val_data = val_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=val_data_txt.column_names,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [24]:
test_data = test_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=test_data_txt.column_names,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [25]:
type(train_data_txt)

datasets.arrow_dataset.Dataset

In [26]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=891f2ab0d39814ea1fbd2df1deee84ad2dcc37fea2a10cbbbcba80235a5c7548
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [27]:
#METRICS
nltk.download("punkt", quiet=True)

metric = datasets.load_metric("rouge")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

In [28]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [29]:
pip install wandb

Note: you may need to restart the kernel to use updated packages.


In [30]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [31]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [32]:
data_collator

DataCollatorForSeq2Seq(tokenizer=T5Tokenizer(name_or_path='t5-base', vocab_size=32100, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>

In [33]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [34]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_NOTEBOOK_NAME"] = 't5model'

In [35]:
#EVALUATE BEFORE FINE TUNING
trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find t5model.


{'eval_loss': 4.004023551940918,
 'eval_rouge1': 31.6882,
 'eval_rouge2': 21.0228,
 'eval_rougeL': 28.4811,
 'eval_rougeLsum': 29.9327,
 'eval_gen_len': 17.736,
 'eval_runtime': 94.5079,
 'eval_samples_per_second': 5.291,
 'eval_steps_per_second': 1.323}

In [36]:
#TRAIN THE MODEL
trainer.train()

Step,Training Loss
50,3.631700
100,3.380300
150,3.095900
200,2.965100
250,2.778800
300,2.774300
350,2.855900
400,2.777800
450,2.849000
500,2.794200


TrainOutput(global_step=3480, training_loss=2.654366580919288, metrics={'train_runtime': 678.7076, 'train_samples_per_second': 20.502, 'train_steps_per_second': 5.127, 'total_flos': 1883281167482880.0, 'train_loss': 2.654366580919288, 'epoch': 5.0})

In [37]:

#EVALUATE AFTER FINE TUNING
trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 2.8057451248168945,
 'eval_rouge1': 46.6567,
 'eval_rouge2': 34.9836,
 'eval_rougeL': 42.4541,
 'eval_rougeLsum': 44.1899,
 'eval_gen_len': 18.352,
 'eval_runtime': 87.4265,
 'eval_samples_per_second': 5.719,
 'eval_steps_per_second': 1.43,
 'epoch': 5.0}

In [38]:
#Generate summaries from the fine-tuned model and compare them with those generated from the original, pre-trained one.
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples['clustersummary'],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

In [39]:
pip install --upgrade transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 78.9 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.16.2
    Uninstalling transformers-4.16.2:
      Successfully uninstalled transformers-4.16.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
simplet5 0.1.4 requires transformers==4.16.2, but you have transformers 4.34.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [41]:
#model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model_before_tuning = T5ForConditionalGeneration.from_pretrained('t5-small')
test_samples = val_data_txt.select(range(5))
model_after_training = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-xsum-12-3")
model_before_tuning = T5ForConditionalGeneration.from_pretrained('t5-small')
summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]
summaries_after_tuning = generate_summary(test_samples, model)[1]
summaries_after_training = generate_summary(test_samples, model_after_training)[1]

In [43]:
summaries_before_tuning

["and CV and it's unlikely to have negative repercussions on them.",
 'that this is a bad decision.',
 '.',
 '.........',
 '. your desk so that there is no space behind you to stand. You could place']

In [44]:
summaries_after_tuning

["It is generally agreed that it's perfectly acceptable to help someone format their CV and it'",
 'It is generally agreed that this is a bad decision. The only situation that this could be',
 'The general consensus is that, whilst this lack of motivation is understandable, it is',
 'The best answer to this question would be to simply explain how your approached the problem, what potential',
 "You could consider angling your desk to make it seem more 'open' to visitors"]

In [45]:
summaries_after_training

["’ compte generally agreed that it's perfectly acceptable to help to helpre.T alreadya deep beBN have itdX",
 'de applicationa. were the only way of were the water way ofensuring that your application arrived on time. applicationa time. wayX',
 'at this lack of motivation is understandable, it is not ethical to not work your notice period after after after at this Gabsignreing. Some people may considerX',
 'ul best answer to heater answer tot this question. besta. best answer. question. simply explain how your approached the problem, what potential solutions you identified and how identified and you chose the one to pursue. This is not generall considered considered and how you choseX',
 'dance considerangling your desk to makeangling. You could place obstacles on the floor, such as plants or Atmosphäreaa backpack to discourage, you could alwaysX']

In [46]:
for i in range(10):
  print(val_data_txt['secondsummary'][i])


It is generally agreed that it's perfectly acceptable to help someone format their CV and it's unlikely to have any negative repercussions for them.
The general consensus is that this is a bad decision unless it is the absolute only way of ensuring that your application arrives on time.
The general consensus is that, whilst this lack of motivation is understandable, this is not ethical behaviour and some people would even consider it to be illegal as a breach of contract.
This sort of question is not generally considered to be a trick question. The most appropriate answer would be simply to describe how you approached a past problem, what solutions you identified and how you chose the one to pursue.
There are a number of options available, including orientating your desk so that there is no need to move behind you or placing objects on the floor, like plants, to obstruct an easy path behind your desk. If people need to see your screen, you could mount it on a rotating arm, and finally,